# Optical Flow

In [1]:
import os
import cv2 as cv
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
N_CLASSES = 7
classes = [str(i) for i in range(N_CLASSES)]

def mk(filename):
    if not os.path.exists(filename):
        os.makedirs(filename, exist_ok=True)

def read_frames(video_filename):
    cap = cv.VideoCapture(video_filename)
    frames = []
    ret, frame = cap.read()
    while ret:
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frames.append(gray)
        ret, frame = cap.read()
    cap.release()
    return frames

def frame_sequence_to_flow(f1, f2):
    flow = cv.calcOpticalFlowFarneback(f1, f2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
    mask = np.zeros((len(f1), len(f1[0]), 3), dtype=np.float32)
    mask[..., 0] = angle * 180 / np.pi / 2
    mask[..., 1] = 255
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
    return rgb

def extract_flow(input_dir, output_dir, c, filename, frame_step):
    in_fullname = os.path.join(input_dir, c, filename)
    out_fullname = os.path.join(output_dir, c, "frame_{}_" + os.path.splitext(filename)[0] + ".jpg")

    print("processing", in_fullname)
    frames = read_frames(in_fullname)
    n = len(frames)
    flow_frame_num = 0

    for i in range(n - frame_step):
        frame1 = frames[i]
        frame2 = frames[i + frame_step]
        flow_frame = frame_sequence_to_flow(frame1, frame2)
        save_path_and_name = out_fullname.format(flow_frame_num)
        flow_frame_num += 1
        cv.imwrite(save_path_and_name, flow_frame)
        
    return in_fullname  # Return the processed filename for verification

def process_videos(input_dir, output_dir, c, filenames, frame_step):
    for filename in filenames:
        if filename.endswith(".mp4"):
            extract_flow(input_dir, output_dir, c, filename, frame_step)

def process_dataset(dataset_dir, FPS=16, frame_step=None, max_workers=8):
    frame_step = frame_step or (FPS // 3)
    executor = ThreadPoolExecutor(max_workers=max_workers)

    for partition in ["test", "trainval"]:
        input_dir = os.path.join(dataset_dir, "videos", partition)
        output_dir = os.path.join(dataset_dir, "of", partition)
        
        mk(output_dir)
        for c in classes:
            mk(os.path.join(output_dir, c))
            class_dir = os.path.join(input_dir, c)
            filenames = os.listdir(class_dir)

            # Using ThreadPoolExecutor to process videos in parallel
            future_to_video = {executor.submit(process_videos, input_dir, output_dir, c, [filename], frame_step): filename for filename in filenames if filename.endswith(".mp4")}
            
            for future in as_completed(future_to_video):
                url = future_to_video[future]
                try:
                    data = future.result()
                except Exception as exc:
                    print('%r generated an exception: %s' % (url, exc))


In [3]:
dataset_dir = 'kaggle-dataset-6classes-preprocessed'
process_dataset(dataset_dir)

processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_001_A_11_G_01.mp4
processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_001_A_12_G_01.mp4
processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_002_A_12_G_01.mp4
processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_003_A_12_G_01.mp4
processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_005_A_12_G_01.mp4
processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_007_A_11_G_01.mp4
processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_007_A_12_G_01.mp4
processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_008_A_12_G_01.mp4
processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_009_A_12_G_02.mp4
processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_011_A_12_G_02.mp4
processing kaggle-dataset-6classes-preprocessed\videos\test\0\HandWash_012_A_11_G_03.mp4
processing kaggle-dat